<div align="center">
  <h1><b>Personal French Vocabulary Learning Assistant</b></h1>
  <p>GooglexKaggle 5 Day Generative AI | Capstone Project | By <a href="https://github.com/ndhng">Hưng Nguyễn</a></p>
  <br> </div>

## **About the project**

This Jupyter Notebook details the development of a Personal French Vocabulary Learning Assistant, the capstone project for the Gen AI Intensive Course - 2025Q1. This project leverages generative AI techniques to create an interactive and personalized tool designed to help users expand their French vocabulary.

--------------

**Motivation:** The goal is to apply learned generative AI skills to create a meaningful and functional application. This project addresses the challenge of effective French language learning by providing a personalized and engaging vocabulary acquisition experience. By tailoring learning to individual levels and preferences, this assistant aims to enhance the efficiency and enjoyment of French vocabulary acquisition.

**Overview:** This notebook outlines the key stages of building the Personal French Vocabulary Learning Assistant:

1.  **Imports and Setup:** Importing essential Python libraries and configuring the environment, including loading the Google API key and initializing global variables.
2.  **Initialization:** Handling the initial state, including defining the workbook's global variables and checking for vocabulary history.
3.  **User State Management:** Defining the `UserState` TypedDict to manage all relevant information about the user's interaction and learning progress during the session.
4.  **Tools:** Implementing a suite of tool functions that the **Learning Assistant Agent** can use to interact with and modify the `UserState`, enabling actions like getting and setting user level, vocabulary, learning mode, and managing the session flow.
5.  **Prompt Library:** Centralizing a system and conversation prompt library to input into each AI prompt.
6.  **Nodes Definition and Flow Controlling:** Designing the different nodes within the interaction process, including the `human_node`, `bot_node`, and `process_tool_calls` nodes; as well as the logic behind routing each node's output.
7.  **Model Initialization and Interaction Process Mapping:** Initializing the Gemini Model and mapping out the process using the nodes and routing defined
8.  **Final Outcome:** An interactive assistant to help you learn or test French vocabulary.

**GenAI Capabilities Implemented:** As a requirement for the capstone project, 3 capabilities was implemented:

1. **Agent:** The entire Assistant itself is the agent
2. **Function Calling:** The Assistant can use a set of predefined functions *(Details in Section 4)*
3. **Few-shot Prompting:** Some few-shot prompting method was implemented to interact with the Gemini API *(Details in Section 5)*


---------------------------------
### **Section 1. Imports and Setup**

This section prepares the Python environment. It involves installing necessary libraries and importing the required modules, as well as setting up the API key for the chosen Generative AI model.

***1.1. Installing Required Libraries:***

The project relies on several key Python libraries. 

*If you are running this Jupyter Notebook locally where you already have the necessary libraries installed, you may comment out the code block below.*

In [ ]:
# Remove conflicting packages from the Kaggle base environment.
# !pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

# Install langgraph and the packages used in this project.
# !pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

***1.2. Importing Libraries:***

This step imports the specific Python libraries and modules that will be utilized throughout the notebook. These imports provide access to functionalities for interacting with Large Language Models, building conversational flows, managing data structures, and more.

In [ ]:
import os

from IPython.display import Image, display, Markdown
from typing import Annotated, List, Literal, TypedDict
from pprint import pprint

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_core.tools import tool

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

***1.3. Set up Google API Key:***

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`. If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

*Alternatively, if you are running this Notebook locally, save your API KEY to an .env file in the same folder and load it using `load_dotenv()`.*

In [ ]:
# # # Option 1: Load the API key from Kaggle secrets
# from kaggle_secrets import UserSecretsClient
# GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Option 2: Load the API key from a .env file
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

---------------------------------
### **Section 2. Initialization**

This section handles the initial setup and defines crucial global variables for the agent. It also addresses the handling of the user's vocabulary history.

***2.1. Defining Global Variables:***

Several global variables are defined at the beginning of the notebook to configure the behavior of the learning assistant. These include:

-   `bot_name`: The name assigned to the learning assistant. You may set whatever name you want the assistant to refer to itself by.
-   `user_name`: The name that gets displayed everytime you chat with the assistant.
-   `recursion_limit`: A setting to manage recursion limit when traversing through the LangGraph flow when interacting with the assistant.
-   `debug`: A boolean flag to enable or disable debugging output during the execution of the notebook. Setting this to `True` will print logs during each step.
-   `llm_model`: A string specifying the specific Google's Large Language Model to be used (e.g., "gemini-2.0-flash").

These global variables provide a central point for configuring various aspects of the assistant's operation.

In [ ]:
# Names
bot_name = "Frenchie"
user_name = "You"

# Maximum Gemini API call limit
recursion_limit = 100

# Debug mode
debug = False

# Gemini model
llm_model = "gemini-2.0-flash"

***2.2. Vocabulary History Handling:***

To personalize the learning experience, the assistant attempts to leverage the user's existing vocabulary history. An empty list, `history_learned_vocab`, is initialized to store this information.

The notebook includes a reminder for the user to update their learning history.
* If the user does not have a vocabulary history yet, they are free to continue with the rest of the Notebook.
* Otherwise, they should re-run the codeblock that they have copied from the Assistant at the end of their previous Jupyter Notebook session. If they fail to do so, the agent would not have any memory of the previous conversation once each Jupyter Notebook session ends.

In [ ]:
history_learned_vocab = []

if not history_learned_vocab:
    input("The vocabulary list is empty. \nIf you have saved a learned_vocab list in previous sessions, remember to run the code block again. \n Otherwise, feel free to continue. ")

---------------------------------
### **Section 3. User State Management**

This section defines the `UserState` that traverses through the LangGraph flow, which stores the state of the user's interaction with the Personal French Vocabulary Learning Assistant. Effective state management is crucial for maintaining context throughout the conversation and personalizing the learning experience.

To structure and type-hint the information related to the user's study conversation, a `TypedDict` class is defined. This dictionary holds various key pieces of information, including:

-   `messages`: A list to store the history of messages exchanged between the user and the assistant.
-   `level`: A string to store the user's determined CEFR French level (e.g., "A1", "B2").
-   `learned_vocab`: A list to keep track of the French vocabulary the user has learned during the session or from previous interactions.
-   `mode`: A string to indicate the current learning mode selected by the user ("learn" or "test").
-   `route_to`: A string used to control the flow of the conversation and direct the next steps, currently determines when to exit the LangGraph flow.
-   `stage`: A string to represent the current stage or step within a specific learning or testing flow, currently determines the conversation prompt the AI receives.

*Note: `route_to` and `stage` currently serve similar functionalities, as originally I planned to make the process flow more complicated, requiring different variables.*

In [ ]:
class UserState(TypedDict):
    """State representing the user's study conversation."""

    # The chat conversation.
    messages: Annotated[List[BaseMessage], add_messages]

    # The user's French CEFR level - A1, A2, B1, B2, C1, C2
    level: str

    # The user's learned vocabulary -> For scalability, this can be changed to "topics_covered" to avoid saving a massive list of vocabulary
    learned_vocab: List[str]

    # The user's mode
    mode: str

    # Route to
    route_to: str

    # Stage
    stage: str

---------------------------------
### **Section 4. Tools**

This section details the implementation of various tool functions that the **Learning Assistant Agent** can utilize to interact with and modify the `UserState`. These tools provide the agent with specific capabilities to manage the user's learning journey and the flow of the conversation. Each tool function is designed to perform a specific action related to the user's state.

***4.1. Tool Functions Schemas:***

The tool functions define here have no functional body within this section of the code. This is because LangGraph does not permit `@tool`-decorated functions to directly update the conversation state. Therefore, they have no body but only docstrings to allow the agent to understand how to use them. 

These empty functions will be bound to the LLM but their implementation is deferred to the `process_tool_calls` node, which will be defined in **Section 4.2**.

In [ ]:
@tool
def get_user_level() -> str:
    """Get the user's CEFR level.
    Returns:
        The user's CEFR level saved currently, or None if no memory was saved.
    """

@tool
def get_learned_vocabulary() -> List[str]:
    """Get the user's learned vocabulary.
    Returns:
        The user's learned vocabulary saved currently, or None if no memory was saved.
    """

@tool
def get_learning_mode() -> str:
    """Get the user's learning mode.
    Returns:
        The user's learning mode saved currently, or None if no memory was saved.
    """

@tool
def set_user_level(level: str) -> str:
    """Set the user's CEFR level as "A1", "A2", "B1", "B2", "C1", or "C2" according to user's response.
    Returns:
        The updated CEFR level of the user, and begin the content generation stage.
    """

@tool
def update_learned_vocabulary(words: List[str]) -> List[str]:
    """Update the user's learned vocabulary with a list of words.
    Args:
        words: A list of words to add to the user's learned vocabulary.
    Returns:
        The updated learned vocabulary of the user.
    """

@tool
def set_learning_mode(mode: str) -> str:
    """Set the user's learning mode as "passive" or "active" according to user's response.
    Returns:
        The updated learning mode of the user.
    """

@tool
def exit_session() -> str:
    """The user is trying to exit the session. Get the learned_vocab list to print the necessary code block.
    Returns:
        The Python code block to initialize learned_vocab list.
    """

@tool
def update_stage(stage: str) -> str:
    """Update the user's stage in the conversation. They can be 'INIT_LEVEL', 'CONTENT', or 'EXIT'.
    Args:
        stage: The new stage to set for the user.
    Returns:
        The updated stage of the user.
    """

tools = [
    get_user_level, 
    get_learned_vocabulary, 
    get_learning_mode, 
    get_user_level, 
    update_learned_vocabulary, 
    set_learning_mode, 
    exit_session, 
    update_stage]

***4.2. Tool Processing Node:***

The actual implementation of how these tools modify the `UserState` will be handled within a separate node in the LangGraph flow, `process_tool_calls`. This separation ensures that state updates are managed explicitly within the graph's logic, rather than implicitly within the tool functions themselves. This approach provides better control and clarity over the state management process, preventing the Language Model from directly and potentially arbitrarily manipulating the application's internal state.

In [ ]:
def process_tool_calls(state: UserState) -> UserState:
    """The user state node. This is where the user state is manipulated."""
    if debug:
        print(f"------ NODE: Processing tool calls")
    
    tool_msg = state.get("messages", [])[-1]
    learned_vocab = state.get("learned_vocab", [])
    mode = state.get("mode", None)
    level = state.get("level", None)
    route_to = state.get("route_to", None)
    stage = state.get("stage", None)
    outbound_msgs = []

    for tool_call in tool_msg.tool_calls:

        if tool_call["name"] == "get_user_level":
            if level is not None:
                response = f"The user's saved CEFR level is {level}."
            else:
                response = "No CEFR level saved yet."
        
        elif tool_call["name"] == "update_stage":
            if tool_call["args"] and isinstance(tool_call["args"]["stage"], str):
                stage = tool_call["args"]["stage"]
                state["stage"] = stage
                response = f"The user's stage has been updated to {stage}."
            else:
                response = "Invalid arguments for update_stage."
        
        elif tool_call["name"] == "get_learned_vocabulary":
            if learned_vocab:
                response = f"The user's learned vocabulary is: {', '.join(learned_vocab)}."
            else:
                response = "No learned vocabulary saved yet."
        
        elif tool_call["name"] == "get_learning_mode":
            if mode is not None:
                response = f"The user's saved learning mode is {mode}."
            else:
                response = "No learning mode saved yet."
        
        elif tool_call["name"] == "set_user_level":
            if tool_call["args"] and isinstance(tool_call["args"]["level"], str):
                level = tool_call["args"]["level"]
                state["level"] = level
                response = f"The user's CEFR level has been set to {level}. If no learning mode is selected yet, ask the user to choose one."
                stage = "CONTENT"
                state["stage"] = stage
            else:
                response = "Invalid arguments for set_user_level."
        
        elif tool_call["name"] == "update_learned_vocabulary":
            if tool_call["args"] and isinstance(tool_call["args"]["words"], list):
                learned_vocab.extend(tool_call["args"]["words"])
                learned_vocab = list(set(learned_vocab))
                state["learned_vocab"] = learned_vocab
                response = f"The user's learned vocabulary is updated to: {', '.join(learned_vocab)}."
            else:
                response = "Invalid arguments for update_learned_vocabulary."
        
        elif tool_call["name"] == "set_learning_mode":
            if tool_call["args"] and isinstance(tool_call["args"]["mode"], str):
                mode = tool_call["args"]["mode"]
                state["mode"] = mode
                response = f"The user's learning mode is set to {mode}."
            else:
                response = "Invalid arguments for set_learning_mode."
        
        elif tool_call["name"] == "exit_session":
            stage = "EXIT"
            state["stage"] = stage
            session_learned_vocab = state.get("learned_vocab", [])
            response = f"The user is trying to exit. The learned vocabulary is: {', '.join(session_learned_vocab)}. Return the code block to save the learned vocabulary."

        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')
        
        # Record the tool results as tool messages.
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "learned_vocab": learned_vocab, "mode": mode, "level": level, "route_to": route_to, "stage": stage}


---------------------------------
### **5. Prompt Library**

This section centralizes the collection of system prompts and conversation prompts that are used as input for the Language Model (LLM) at various stages of the interaction. Maintaining a dedicated prompt library enhances organization, facilitates experimentation, and ensures consistency in how the LLM is instructed throughout the conversation.

***5.1. System Prompt:***

The system prompt defines the overall behavior and persona of the Learning Assistant Agent. This provides the chatbot with essential guidance on its behavior, specifying when to utilize different functions and establishing conversational norms regarding tone and permissible discussion areas.

In [ ]:
# The system instruction defines how the chatbot is expected to behave and includes
# rules for when to call different functions, as well as rules for the conversation, such
# as tone and what is permitted for discussion.
SYSINT = (
    "system",
    f"""You are {bot_name}, an interactive French vocabulary learning assistant for an English-speaking user. Your primary goal is to help users expand their French vocabulary based on their specified language level and learning preferences.

Say hello if it is the first interaction. You will then interact with the user to understand their language proficiency and learning demand. Once you've understood the user and their study mode, you will then generate relevant learning or testing content and guide them through the session.
You have access to tools to:
- Get and update the user's French CEFR level (`get_user_level`, `set_user_level`).
- Get and update the user's learned vocabulary (`get_learned_vocabulary`, `update_learned_vocabulary`).
- Get and set the current learning mode (`get_learning_mode`, `set_learning_mode`).
- Exit the session (`exit_session`).
- Update the user's stage in the conversation (`update_stage`). You only need to update the stage to "EXIT" when the user is trying to exit the session.

Remember to only discuss French vocabulary learning and stay strictly in the current step. Avoid making any assumptions about the user's intent at all costs. If a requested tool is unavailable, inform the user. Be mindful of the overall length of the conversation.
At any step, if it seems like the user is trying to leave, attempt to go through the exiting procedure.
"""
)

***5.2. Conversation Prompts:***

Conversation prompts are used during the ongoing interaction to elicit specific responses or guide the LLM's output based on the current state and user input. These prompts are organized into a dictionary, with keys being the predefined stages of the `UserState`. 

Benefits:
* Making updating the instructions given to the LLM during the prompting step easier, contributing to a more robust and well-defined conversational flow. 
* Centralizing all the AI prompts into one place for easier review and update when needed.

The specific content of these prompts is crucial in shaping the assistant's behavior and the quality of the learning experience.

In [ ]:
CONVERSATION_PROMPTS = {
    "WELCOME": """
You are now initializing the session. Say hello and welcome in French and introduce yourself in an enthusiastic way. 
Instruct the user to type `quit` to quit if they want. 
Ask the user to describe their current French level and desired learning mode.
""",

    "INIT_LEVEL": """
TASK: Understand the user's response regarding their CEFR level and update the user's level.
INSTRUCTIONS:
- You will receive the user's response when they are asked to describe their French level. It might be a simple answer like "A1" or "B2", a more complex one like "I am a beginner" or "I can read and write in French", or their confirmation regarding your previous conversations.
FEW SHOTS EXAMPLES:
+ "I am a beginner" -> Update user level to A1
+ "I can read and write fluently in French and I prefer to study actively" -> Update user level to B2 and mode to active
+ "I prefer to study passively" -> Update user mode to passive and ask the user to confirm their level
- You will then need to determine the user's CEFR level from their response. If uncertain, ask the user if what you think is correct.
- If the detected CEFR level and/or the study mode is confirmed, update each/both accordingly, otherwise, ask the user to clarify their level.
- You may repeat the question if the user refuses to or cannot provide a sufficient answer.
""",

    "CONTENT": """
TASK: Generate content based on the user's level, learning mode, and learned vocabulary.
INSTRUCTIONS:
- If the user has not set their learning mode yet, ask them to choose between passive and active learning.
- The content generated should be relevant to the user's level, study mode, and existing vocabulary.
- Based on the chat history, you may: 
+ For passive learning mode, generate content to teach new words, explain meaning and use in context. Avoid re-teaching previously learned vocabulary and focus on new words.
+ For active learning mode, generate a test or quiz based on the user's level and previously learned vocabulary. If the user has learned little to no vocabulary saved, you may then generate a quiz based on the user's level outside of that list.
+ Answer the user's questions and evaluate the user's answers during the lesson when applicable.
- After each successful interaction (new word learned, test passed, etc.), update the user's learned vocabulary. This should include all the words learned, not just the specified vocabulary. Do it quietly, without asking the user for confirmation.
- When asked, update the user's learning mode and CEFR level.
- If the user is trying to exit and has confirmed that is their intent, perform the tool call 'exit_session' and say nothing. 
""",

    "EXIT": """
TASK: The user has indicated they want to quit the conversation. Generate the updated vocab list for the user to copy into this file so that it can be saved for future sessions.
INSTRUCTIONS:
- 1: Generate a Python code that contains the learned vocabulary list, always named "history_learned_vocab".
- 2: The output should be nothing else but the code block, which can use Markdown syntax highlighting.
- 3: The code block should contain the learned vocabulary list, which is a list of strings.
- 4: The code block should be formatted in a way that the user can copy and paste it into their Python environment.
- 5: No other text should be included in the code block.
CODE BLOCK EXAMPLE:
```
# Copy and paste this code block into your Jupyter Notebook or Python environment to save your learned vocabulary.
history_learned_vocab = ["Bonjour", "Merci", "Au revoir"]
```
"""
}

UNKNOWN_STAGE_PROMPT = """
TASK: Process the user's previous input and determine the next step."""

---------------------------------
### **Section 6. Nodes Definition and Flow Controlling**

Beside the `process_tool_calls` node defined in **Section 4**, this section details the design and implementation of the other two nodes (`human_node` and `bot_node`), which constitute the interaction process within the Learning Assistant. It also explains the logic behind routing the output of each node to determine the subsequent steps in the conversation flow. The architecture leverages LangGraph to create a stateful and dynamic interaction.

***6.1. Node Definitions:***

The interaction process is composed of several distinct nodes, each responsible for a specific task:

-   **`bot_node`:** This node represents the Learning Assistant Agent. It uses the Language Model, along with the defined tools and prompts, to generate the next response or decide on the next action (which might involve calling a tool). The `UserState`'s `stage` is used to determine part of the logic within the bot. Once the bot has generated a chat response, it also gets displayed to the user.

-   **`human_node`:** This node handles receiving input directly from the user, then displays the user's input and updates the `UserState` with the new user message.

In [ ]:
def bot_node(state: UserState) -> UserState:
    """The AI assistant with tools, responsible for generating content, making tool calls, or both."""
    if debug:
        print(f"------ NODE: Chatbot")
    defaults = {"stage": "INIT_LEVEL", "learned_vocab": history_learned_vocab}
    if state["messages"]:
        current_stage = state["stage"]
        if current_stage not in CONVERSATION_PROMPTS:
            if debug:
                print(f"------ Unknown stage: {current_stage}, setting to UNKNOWN_STAGE.")
            current_stage = "UNKNOWN_STAGE"
        conversation_prompt = CONVERSATION_PROMPTS.get(current_stage)
        learned_vocab = state.get("learned_vocab", [])
        learned_vocab_prompt = f"User's learned vocabulary is: {', '.join(learned_vocab)}"
        if debug:
            print(f"------ Prompting AI for stage: {current_stage} with messages history, learned vocab, and prompt: {conversation_prompt}") #### Bug 1: Successfully printed
        new_output = llm.invoke([SYSINT, conversation_prompt, learned_vocab_prompt] + state["messages"])
        if debug:
            print("------ AI response (Condition #2): ", new_output) #### Bug 1: Didn't print until after human's new input, although the new_output was generated before the human input
    else:
        conversation_prompt = CONVERSATION_PROMPTS["WELCOME"]
        if debug:
            print("------ Prompting AI to greet the user...")
        new_output = llm.invoke([SYSINT, conversation_prompt])
        if debug:
            print("------ AI greeting message (Condition #1): ", new_output)

    # Bug 2: Return immediately if the output is not a tool call. This implementation currently returns the AI's response twice if both a response and a tool call is generated, looping the agent back to the bot_node.
    if (hasattr(new_output, "content") and len(new_output.content) > 0):
        print(f'********** {bot_name} **********')
        print(new_output.content)

    return defaults | state | {"messages": [new_output]}

def human_node(state: UserState) -> UserState:
    """Receive the user's input to be fed into the bot node and handle force quit.""" #The only point receiving user input.
    if debug:
        print(f"------ NODE: Human node")

    response = input(f"{user_name}: ")
    if response in {"force quit"}:
        state["route_to"] = "end_node"
        return state
    
    print(f'********** {user_name} **********')
    print(response)
    return state | {"messages": [("user", response)]}


---------------------------------
***6.2. Flow Controlling (Routing Logic):***

The flow of the conversation between these nodes is controlled by routing logic implemented using LangGraph's conditional edges. 

* After each `bot_node` response, the `bot_response_routing` determines whether the next step should require the user's input, make a tool call, or exit altogether.
* After each `human_node` input, the `human_response_routing` determines if it is a force quit command (currently is the string `"force quit"`) and then try to exit. Otherwise, it will feed back into the `bot_node` for processing.

This node-based architecture with explicit flow control allows for a structured and adaptable conversational experience, where the Learning Assistant Agent can interact with the user, utilize tools to manage state, and progress through different stages of the learning process.

In [ ]:
def bot_response_routing(state: UserState) -> Literal["human_node", "process_tool_calls", "__end__"]:
    """Route between chat and tool nodes if a tool call is made."""
    if debug:
        print(f"------ ROUTING NODE: Maybe route to tools")
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("route_to", False) == "end_node" or state["stage"] == "EXIT":
        if debug:
            print(f"------ Routed to END")
        print(f"{bot_name}: Au revoir et à bientôt, paste the returned code block into your Python environment to save your learned vocabulary.")
        return END
    
    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        if debug:
            print(f"------ Routed to process_tool_calls")
        return "process_tool_calls"

    else:
        if debug:
            print(f"------ Routed to human_node")
        return "human_node"
    
def human_response_routing(state: UserState) -> Literal["bot_node", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if debug:
        print(f"------ ROUTING NODE: Maybe exit human node")
    if state.get("route_to", False) == "end_node": # force quitting will happen here
        if debug:
            print(f"------ Routed to END")
        return END
    else:
        if debug:
            print(f"------ Routed to bot_node")
        return "bot_node"

---------------------------------
### **Section 7. Model Initialization and Interaction Process Mapping**

This section details how the interaction process is mapped out and implemented using the nodes and routing logic defined in the previous section. It covers the initialization of the Language Model, the binding of tools, and the creation of the LangGraph that orchestrates the conversation flow.

**7.1. Initializing the Language Model and Binding Tools:**

The foundation of the Learning Assistant Agent is the Large Language Model (LLM). In this implementation, the Gemini model is initialized with specific parameters:

-   `model`: Specifies the Gemini model to be used (configured by the `llm_model` global variable).
-   `temperature`: Controls the randomness of the model's output. A value of 0.5 balances creativity and coherence.
-   `max_tokens`: Sets the maximum number of tokens the model can generate in a single response.
-   `google_api_key`: Authenticates access to the Google AI API.

The defined tool functions (from the "Tools" section) are then bound to this LLM instance. This binding allows the LLM to understand and call these tools during the conversation when necessary. A `RunnableConfig` is also set, including a `recursion_limit` to prevent excessively deep conversational loops.

In [ ]:
base_llm = ChatGoogleGenerativeAI(
    model=llm_model, 
    temperature=0.5, 
    max_tokens=2000, 
    google_api_key=GOOGLE_API_KEY
    )

llm = base_llm.bind_tools(tools)
config = RunnableConfig(recursion_limit=recursion_limit)

***7.2. Interaction Process Mapping***

The interaction is built as a state graph where each node represents a specific function in the conversation (e.g., interacting with the bot, receiving user input, processing tool calls). Edges define the possible transitions between these nodes, creating the overall conversational pathway. Conditional edges introduce logic to dynamically route the conversation based on the output of the nodes.

Specifically, the graph defines nodes for the bot's responses, handling user input, and processing any tool calls generated by the bot. The flow begins with the bot, and subsequent turns are determined by routing functions that analyze the bot's and user's responses to decide the next appropriate node in the interaction. This framework allows for a flexible and responsive conversational experience.

In [ ]:
# Define the LangGraph state graph with the UserState
graph_builder = StateGraph(UserState)

# Add the different nodes to the graph
graph_builder.add_node("bot_node", bot_node)
graph_builder.add_node("human_node", human_node)
graph_builder.add_node("process_tool_calls", process_tool_calls)

# Define the initial edge from the start of the graph to the bot's first turn
graph_builder.add_edge(START, "bot_node")

# The process_tool_calls will always return to the bot_node
graph_builder.add_edge("process_tool_calls", "bot_node")

# The routing edges will determine the next node bot_node and human_node will go to
graph_builder.add_conditional_edges("bot_node", bot_response_routing)
graph_builder.add_conditional_edges("human_node", human_response_routing)

# Compile the graph to create an executable runnable
graph_builder = graph_builder.compile()

# Optional: Visualize the graph (requires graphviz and potentially other dependencies)
# Image(graph_builder.get_graph().draw_mermaid_png())

---------------------------------
### **Section 8. Final Outcome**

Finally, you can run the following code block to interact with the Personal French Vocabulary Learning Assistant built in this Jupyter Notebook. By leveraging the power of generative AI through the Gemini model and orchestrating the conversation flow with LangGraph, this assistant provides a dynamic and potentially personalized learning experience for French vocabulary acquisition.

The assistant is designed to:

-   Engage the user in a conversation to determine their current French language proficiency level.
-   Offer different study modes, allowing the user to either learn new vocabulary or test their existing knowledge.
-   Utilize the Language Model to generate relevant vocabulary and create learning or testing materials tailored to the user's level.
-   Process user input and guide the conversation through defined stages using a stateful graph architecture.
-   Employ tools to manage the user's learning state, such as tracking their level and learned vocabulary.

+   **After each session, the agent will return a Python code to save the user's learned vocabulary list, which can be saved in Section 2 to save progress.**

Through this interactive process, the Personal French Vocabulary Learning Assistant aims to be a helpful tool for individuals seeking to expand their French vocabulary in an engaging and adaptive manner.

In [ ]:
state = graph_builder.invoke({"messages": []}, config=config)

if debug:
    pprint(state)

***Note**: Remember to save the `history_learn_vocab` list that gets returned at the end of each session to save your progress!*

---------------------------------
## **Conclusion**

This Jupyter Notebook has demonstrated the construction of a Personal French Vocabulary Learning Assistant powered by generative AI. By leveraging the capabilities of a Large Language Model and orchestrating the conversational flow with LangGraph, this project showcases a potential approach to creating interactive and adaptive language learning tools.

The implemented assistant guides users through initial level assessment, offers a choice between learning new vocabulary and testing existing knowledge, and utilizes the AI model to generate relevant content and manage the interaction. The modular design, utilizing distinct nodes for different conversational functions and explicit routing logic, provides a flexible framework for future expansion and refinement.

While this notebook represents a significant step towards a personalized learning experience, further development could enhance its capabilities. Potential improvements include more sophisticated user profiling, integration of spaced repetition techniques, support for diverse learning preferences, and more comprehensive feedback mechanisms.

Ultimately, this project highlights the potential of generative AI to create engaging and tailored educational applications, offering a glimpse into the future of personalized language learning.

*- [Hưng Nguyễn](https://github.com/ndhng)*